# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### H2 in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2.n_orbitals))

E_FCI = h2.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 4 

FCI energy: -0.9360549199436612


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [4]:
H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 15 terms


Number of UCCSD amplitudes: 1 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 32 expectationvalues

active variables : 1

E=-0.70294360  angles= {(1, 0, 1, 0): 0.0}  samples= None
E=-0.92698008  angles= {(1, 0, 1, 0): -0.5644201636314392}  samples= None
E=-0.93354955  angles= {(1, 0, 1, 0): -0.756477668453395}  samples= None
E=-0.93605486  angles= {(1, 0, 1, 0): -0.6900732739717247}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904080632337539}  samples= None
E=-0.93605492  angles= {(1, 0, 1, 0): -0.6904070264134937}  samples= None
Optimization terminated successfully.
         Current function value: -0.936055
         Iterations: 5
         Function evaluations: 6
         Gradient evaluations: 6

Obtained UCCSD energy: -0.9

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### H2O in 6-31G basis

Now let us try a larger problem of H$_2$O in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [ ]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h2o.n_orbitals))

E_FCI = h2o.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [ ]:
H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h2o.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4)

print('\nObtained UCCSD energy: {}'.format(result.energy))

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [ ]:
#n=10
#result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### H2 in STO-3G basis

Below we perform the entangler screening protocol for H2 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [5]:
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*h2.n_orbitals, h2.n_electrons)

H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 15 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.2822

Selected entanglers:
1.0 [X0 Y1 X2 X3]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [6]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 18 expectationvalues

active variables : 9

E=-0.70294360  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.0}  samples= None
E=-0.77938639  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 0.2822100520133972}  samples= None
E=-0.93592356  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, gamma_2: 0.0, beta_3: 0.0, gamma_3: 0.0, tau_0: 1.411050260066986}  samples= None
E=-0.93603455  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 0.0, g

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### H2O in 6-31G basis

Let us move on to the problem of H$_2$O in 6-31G basis using the same active space as the UCCSD example.

In [7]:
xyz_data = get_molecular_data('h2o', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
hf_reference = hf_occ(2*h2o.n_orbitals, h2o.n_electrons)


H = h2o.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2o.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2o.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.

Hamiltonian has 185 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.0758
2 : 0.0371
3 : 0.0371
4 : 0.031
5 : 0.0301
6 : 0.0249
7 : 0.0198
8 : 0.0198
9 : 0.0173
10 : 0.0173
11 : 0.0
12 : 0.0

Selected entanglers:
1.0 [X0 Y1 X6 X7]
1.0 [X0 Y3 X5 X6]
1.0 [X1 Y2 X4 X7]
1.0 [X2 Y3 X4 X5]
1.0 [X0 Y1 X4 X5]
1.0 [X2 Y3 X6 X7]


With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [8]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2o.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-75.98071144  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0, tau_5: 0.0}  samples= None
E=-75.98783364  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.07584381103515625, 

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [9]:
n = 1
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

E_FCI = h2o.compute_energy(method='fci')

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-74.43748738  angles= {beta_0: 5.320149394895053, gamma_0: 2.5859452883985776, beta_1: 3.4083353727627923, gamma_1: 3.344281811437862, beta_2: 1.6134407215884572, gamma_2: 5.618693061365673, beta_3: 5.654348105217544, gamma_3: 3.504154165506626, beta_4: 1.6332826269055352, gamma_4: 5.697698883307548, beta_5: 6.045395128321983, gamma_5: 6.184976471948195, beta_6: 2.653513459816145, gamma_6: 1.7857608794352495, beta_7: 5.112343103876155, gamma_7: 3.6156935707636704, tau_0: 4.234019638864681, tau_1: 3.948210212640859, tau_2: 2.2461738274448497, tau_3: 1.837032289195928, tau_4: 0.3863865887103037, tau_5: 0.37897790407856574}  samples= None
E=-74.58217295  angles= {beta_0: 5.272259685422397, gamma_0: 2.

E=-75.54320860  angles= {beta_0: 3.093021828585536, gamma_0: 2.448281713054551, beta_1: 3.082479333389019, gamma_1: 3.2014700487841123, beta_2: 3.119887030506886, gamma_2: 5.477699103646955, beta_3: 5.17020009879323, gamma_3: 3.565273017413072, beta_4: 0.8647541780094752, gamma_4: 4.98977368030038, beta_5: 6.35999281713991, gamma_5: 6.359814891095457, beta_6: 2.1321326941486993, gamma_6: 1.8363910425405858, beta_7: 6.242807077720001, gamma_7: 3.9356582625163017, tau_0: 3.111649338061929, tau_1: 2.9929389370829624, tau_2: 3.061521554446774, tau_3: 2.9893672515826255, tau_4: -0.027474990351419426, tau_5: 0.05037073915948967}  samples= None
E=-75.57375129  angles= {beta_0: 3.123269386954152, gamma_0: 2.4324997696769026, beta_1: 3.020718982368158, gamma_1: 3.198599301079864, beta_2: 3.1681729114689583, gamma_2: 5.465813959713354, beta_3: 4.946011714265944, gamma_3: 3.564247889412894, beta_4: 0.34932605457770527, gamma_4: 4.99114917436966, beta_5: 6.419525325254349, gamma_5: 6.3779965864895

E=-75.98851789  angles= {beta_0: 3.139760500866712, gamma_0: 2.4082368581249165, beta_1: 3.1394766984680302, gamma_1: 3.186963008077567, beta_2: 3.141251451838465, gamma_2: 5.434863126102812, beta_3: 3.136366987043268, gamma_3: 3.583982779708434, beta_4: 0.0076219495634775, gamma_4: 4.963771302580159, beta_5: 6.2536686981842635, gamma_5: 6.3746840010205, beta_6: 0.006265856026007962, gamma_6: 1.866553725957192, beta_7: 6.286491568977504, gamma_7: 3.967016479542079, tau_0: 3.0441621497842917, tau_1: 3.1975309534666034, tau_2: 3.080594699263111, tau_3: 3.1889271293628334, tau_4: 0.04180721184641065, tau_5: 0.03542874436652086}  samples= None
E=-75.98856463  angles= {beta_0: 3.141664901216117, gamma_0: 2.4080114754008903, beta_1: 3.144617052028015, gamma_1: 3.1870754879287184, beta_2: 3.142557420817982, gamma_2: 5.434764180684113, beta_3: 3.141907316109345, gamma_3: 3.583862948123763, beta_4: 0.00033592372608055344, gamma_4: 4.9643821305651095, beta_5: 6.266351250254893, gamma_5: 6.374944

# LiH in STO-3G basis

In [10]:
xyz_data = get_molecular_data('lih', geometry=1, xyz_format=True)

basis = 'sto-3g'

lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis)
hf_reference = hf_occ(2*lih.n_orbitals, lih.n_electrons)


H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)


Hamiltonian has 631 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1224
2 : 0.0289
3 : 0.0289
4 : 0.0272
5 : 0.0272
6 : 0.0195
7 : 0.013
8 : 0.013
9 : 0.0099
10 : 0.0099
11 : 0.0095
12 : 0.0092
13 : 0.0092
14 : 0.0088
15 : 0.0088
16 : 0.0084
17 : 0.0084
18 : 0.0083
19 : 0.0083
20 : 0.0083
21 : 0.0083
22 : 0.0071
23 : 0.0041
24 : 0.0041
25 : 0.0006
26 : 0.0006
27 : 0.0
28 : 0.0
29 : 0.0
30 : 0.0
31 : 0.0
32 : 0.0
33 : 0.0
34 : 0.0

Selected entanglers:
1.0 [X2 Y3 X10 X11]
1.0 [X2 Y3 X5 X10]
1.0 [X2 Y3 X4 X11]
1.0 [X2 Y3 X6 X7]
1.0 [X2 Y3 X8 X9]
1.0 [X0 Y1 X4 X5]


In [11]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 60 expectationvalues

active variables : 30

E=-6.10160502  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0, tau_5: 0.0}  samples= None
E=-6.10296663  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0

E=-6.11035790  angles= {beta_0: 3.141593122924504, gamma_0: 0.0, beta_1: 3.141593122924504, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.1485783776728144, tau_1: 0.023625385886928026, tau_2: -0.023625385886928026, tau_3: 0.36175047921382447, tau_4: 0.36175047921382447, tau_5: -0.013594863106660214}  samples= None
E=-6.11042185  angles= {beta_0: 3.1415931951105613, gamma_0: 0.0, beta_1: 3.1415931951105613, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592731799198, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.14159265358979

E=-6.41278480  angles= {beta_0: 3.1416016990694176, gamma_0: 0.0, beta_1: 3.1416005014150983, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.1415989314980606, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 2.787429559827647, tau_1: 0.3987392262631616, tau_2: -0.39873922626315345, tau_3: 1.7581379785708475, tau_4: 1.758137978569096, tau_5: -0.4280384197568744}  samples= None
E=-6.41289583  angles= {beta_0: 3.141601697112696, gamma_0: 0.0, beta_1: 3.141600499386707, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.1415989313486676, gamma_3: 0.0, beta_4: 6.172620726276823e-10, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141

E=-6.88072981  angles= {beta_0: 3.1415940729828566, gamma_0: 0.0, beta_1: 3.14159082411377, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592155920962, gamma_3: 0.0, beta_4: 8.265967936099749e-06, gamma_4: 0.0, beta_5: 2.0401812668816067e-06, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592061383439, gamma_10: 0.0, beta_11: 3.141592061383439, gamma_11: 0.0, tau_0: 3.1641498555105447, tau_1: -0.006163895772877473, tau_2: 0.006163895771882474, tau_3: 1.5731442518730447, tau_4: 1.5731442518729077, tau_5: -0.027804192111929846}  samples= None
E=-6.88235759  angles= {beta_0: 3.141593191306515, gamma_0: 0.0, beta_1: 3.1415915424662924, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.1415915190808743, gamma_3: 0.0, beta_4: 7.874928949847019e-06, gamma_4: 0.0, beta_5: 2.0903617534602053e-06, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8:

In [12]:
n = 1
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

E_FCI = h2o.compute_energy(method='fci')

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 60 expectationvalues

active variables : 30

E=-4.57070324  angles= {beta_0: 2.449222299507639, gamma_0: 5.536592021494485, beta_1: 4.087880029276752, gamma_1: 2.8515544403381754, beta_2: 2.453636645490613, gamma_2: 1.8473067895532653, beta_3: 0.15195661230681323, gamma_3: 3.1879946630318883, beta_4: 3.7959328555969294, gamma_4: 3.054865695375211, beta_5: 3.525589440364006, gamma_5: 1.1670232735865487, beta_6: 1.3278650995730992, gamma_6: 1.0447433033995739, beta_7: 1.7558987831591157, gamma_7: 5.586686625653792, beta_8: 5.443715645482963, gamma_8: 0.8839188393282897, beta_9: 6.024672407633531, gamma_9: 5.843703047461389, beta_10: 4.386905349513471, gamma_10: 1.3296661704928103, beta_11: 4.267604108123005, gamma_11: 3.19022118737895, tau_0: 4.7

E=-7.50504765  angles= {beta_0: 3.1419418053492643, gamma_0: 5.588585791867792, beta_1: 3.1406864725013874, gamma_1: 2.888445618219352, beta_2: 2.5535843600995145, gamma_2: 1.7815790876317774, beta_3: -0.0046760704798004815, gamma_3: 3.157268955200992, beta_4: 3.703663696055699, gamma_4: 2.0317194126473663, beta_5: 3.1797568349353447, gamma_5: 1.160689365527691, beta_6: 1.605883401415439, gamma_6: 1.1365218042307959, beta_7: 2.485636652758111, gamma_7: 5.702928651981618, beta_8: 6.133211638416455, gamma_8: 0.8880399901234584, beta_9: 6.2646461796451565, gamma_9: 5.850949483763661, beta_10: 3.130763224563946, gamma_10: 1.5447866310596838, beta_11: 3.065573289822069, gamma_11: 2.991646665729899, tau_0: 6.301515533792217, tau_1: 3.149701963954578, tau_2: 3.0942457679340722, tau_3: 3.7465758437812533, tau_4: 0.11165214741610965, tau_5: 6.294194870589648}  samples= None
E=-7.50946281  angles= {beta_0: 3.14965574264644, gamma_0: 5.5886676348830235, beta_1: 3.1308654905891666, gamma_1: 2.8885

E=-7.66361506  angles= {beta_0: 3.1179108775439675, gamma_0: 5.587207443153325, beta_1: 3.1766568320552344, gamma_1: 2.887206421412879, beta_2: 0.2477203785715275, gamma_2: 1.7598382053616684, beta_3: 0.02934701155612455, gamma_3: 3.1257040967869183, beta_4: 3.1058170062354646, gamma_4: 1.9627398726376735, beta_5: 3.141991363045754, gamma_5: 1.160259548727333, beta_6: 1.292724484374931, gamma_6: 1.1474717689854779, beta_7: 3.3416874066867304, gamma_7: 5.732460102723089, beta_8: 6.311040676267088, gamma_8: 0.8884815554230705, beta_9: 6.28152527775107, gamma_9: 5.851982727623983, beta_10: 3.2566414810196562, gamma_10: 1.5535679855060198, beta_11: 3.107475467646097, gamma_11: 2.9774129308430872, tau_0: 6.551563947510789, tau_1: 3.1124882075566687, tau_2: 3.067509619270567, tau_3: 3.152531884689411, tau_4: 0.19556344418378566, tau_5: 6.3134631232853}  samples= None
E=-7.68787570  angles= {beta_0: 3.1440015232387095, gamma_0: 5.587494803952097, beta_1: 3.134410727770569, gamma_1: 2.88745332

E=-7.78304744  angles= {beta_0: 3.1443081794519903, gamma_0: 5.58978628396343, beta_1: 3.137830358699336, gamma_1: 2.8887390967965905, beta_2: 0.007028274491904792, gamma_2: 1.7703192806913772, beta_3: -0.005242272961630358, gamma_3: 3.1281475588734886, beta_4: 3.0883896473114727, gamma_4: 1.9473231941234337, beta_5: 3.1053554874609897, gamma_5: 1.1609715543065413, beta_6: 3.137510927684763, gamma_6: 1.1480379245246286, beta_7: 3.0679112990983994, gamma_7: 5.73098358148335, beta_8: 6.258692205507544, gamma_8: 0.8884973541846667, beta_9: 6.2820179578978355, gamma_9: 5.851980249738052, beta_10: 3.151120043192586, gamma_10: 1.56254043279075, beta_11: 3.137705618256683, gamma_11: 2.9743214511462557, tau_0: 6.469978965148574, tau_1: 3.2054043856490315, tau_2: 3.0624101550993372, tau_3: 3.201565421016683, tau_4: 0.0791744851699312, tau_5: 6.287908727972957}  samples= None
E=-7.78334324  angles= {beta_0: 3.142872406001086, gamma_0: 5.5898493616267535, beta_1: 3.1400279346402216, gamma_1: 2.88

E=-7.78350695  angles= {beta_0: 3.141472244613611, gamma_0: 5.58979662418759, beta_1: 3.141773148330765, gamma_1: 2.888712788620501, beta_2: 0.0005266650669252075, gamma_2: 1.7697809741218389, beta_3: 0.002620821239412843, gamma_3: 3.1277811288752324, beta_4: 3.13936315371195, gamma_4: 1.9448358642481105, beta_5: 3.141074597073948, gamma_5: 1.1609501580460169, beta_6: 3.1424347825580594, gamma_6: 1.1484100278419156, beta_7: 3.142721546905003, gamma_7: 5.731577300711183, beta_8: 6.282432842828581, gamma_8: 0.8885300982989651, beta_9: 6.282698125586214, gamma_9: 5.852025522832945, beta_10: 3.141434782751063, gamma_10: 1.563044227983174, beta_11: 3.14120610421734, gamma_11: 2.973693156348837, tau_0: 6.478657361860835, tau_1: 3.2115445221047807, tau_2: 3.0709028277438875, tau_3: 3.210503989879965, tau_4: 0.0696260421540084, tau_5: 6.290525474627021}  samples= None
E=-7.78350759  angles= {beta_0: 3.141506739131973, gamma_0: 5.589793864667089, beta_1: 3.1417174537143264, gamma_1: 2.888710794

# N2 in STO-3G basis

In [13]:
xyz_data = get_molecular_data('n2', geometry=1.2, xyz_format=True)

basis = 'sto-3g'
n2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis)
hf_reference = hf_occ(2*n2.n_orbitals, n2.n_electrons)


H = n2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*n2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*n2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)


Hamiltonian has 2239 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1967
2 : 0.1967
3 : 0.1583
4 : 0.1583
5 : 0.1391
6 : 0.1391
7 : 0.1059
8 : 0.1059
9 : 0.1059
10 : 0.1059
11 : 0.1049
12 : 0.1049
13 : 0.1049
14 : 0.1049
15 : 0.1049
16 : 0.1041
17 : 0.0997
18 : 0.0997
19 : 0.0997
20 : 0.0997
21 : 0.0824
22 : 0.0824
23 : 0.067
24 : 0.067
25 : 0.067
26 : 0.067
27 : 0.0621
28 : 0.0621
29 : 0.0513
30 : 0.0473
31 : 0.0473
32 : 0.0353
33 : 0.0353
34 : 0.0335
35 : 0.0335
36 : 0.0333
37 : 0.0327
38 : 0.0327
39 : 0.0327
40 : 0.0327
41 : 0.0324
42 : 0.0243
43 : 0.0243
44 : 0.0243
45 : 0.0243
46 : 0.0192
47 : 0.0192
48 : 0.0192
49 : 0.0192
50 : 0.0182
51 : 0.0182
52 : 0.0182
53 : 0.0182
54 : 0.0182
55 : 0.0182
56 : 0.0182
57 : 0.0182
58 : 0.0143
59 : 0.0143
60 : 0.0139
61 : 0.0139
62 : 0.0136
63 : 0.0136
64 : 0.0136
65 : 0.0136
66 : 0.0133
67 : 0.0133
68 : 0.0092
69 : 0.0092
70 : 0.0092
71 : 0.0092
72 : 0.009
73 : 0.009
74 : 0.009
75 : 0.009
76 : 0.0087
77 : 0.0087
78

In [ ]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*n2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 92 expectationvalues

active variables : 46

E=-96.01373465  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 3.141592653589793, gamma_4: 0.0, beta_5: 3.141592653589793, gamma_5: 0.0, beta_6: 3.141592653589793, gamma_6: 0.0, beta_7: 3.141592653589793, gamma_7: 0.0, beta_8: 3.141592653589793, gamma_8: 0.0, beta_9: 3.141592653589793, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, beta_12: 3.141592653589793, gamma_12: 0.0, beta_13: 3.141592653589793, gamma_13: 0.0, beta_14: 3.141592653589793, gamma_14: 0.0, beta_15: 3.141592653589793, gamma_15: 0.0, beta_16: 3.14159

In [ ]:
n = 1
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

E_FCI = h2o.compute_energy(method='fci')